# Soluções Avançadas

In [2]:

print("🚀 INICIANDO SOLUÇÕES AVANÇADAS PARA EXTRAÇÃO DE MATERIAIS")
print("=" * 65)

# Configuração inicial
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
from PyPDF2 import PdfReader
from collections import Counter

# Tentar importar bibliotecas opcionais
try:
    import pdfplumber
    PDFPLUMBER_AVAILABLE = True
    print("✅ pdfplumber disponível")
except ImportError:
    PDFPLUMBER_AVAILABLE = False
    print("⚠️  pdfplumber não disponível - usando PyPDF2 como fallback")

try:
    import fitz  # PyMuPDF
    FITZ_AVAILABLE = True
    print("✅ PyMuPDF disponível")
except ImportError:
    FITZ_AVAILABLE = False
    print("⚠️  PyMuPDF não disponível")

# Configurações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

sys.path.append('..')

# Verificar arquivos disponíveis
pasta_projetos = "data/raw"
pasta_path = Path(pasta_projetos)

if not pasta_path.exists():
    caminho_absoluto = r"C:\Users\User\OneDrive - Questindustries\Projetos Python\Plataforma Quantitativos\data\raw"
    pasta_path = Path(caminho_absoluto)
    if pasta_path.exists():
        pasta_projetos = caminho_absoluto

pdfs = list(pasta_path.glob("*.pdf"))
print(f"📁 PDFs disponíveis: {len(pdfs)}")

🚀 INICIANDO SOLUÇÕES AVANÇADAS PARA EXTRAÇÃO DE MATERIAIS
⚠️  pdfplumber não disponível - usando PyPDF2 como fallback
⚠️  PyMuPDF não disponível
📁 PDFs disponíveis: 16


# 🎯 ANÁLISE ESTRUTURAL MANUAL DETALHADA

In [3]:
def analise_estrutural_manual(pdf_path, max_paginas=3):
    """Análise manual detalhada da estrutura do PDF"""
    
    print(f"\n🔍 ANÁLISE ESTRUTURAL MANUAL: {pdf_path.name}")
    print("=" * 50)
    
    try:
        if PDFPLUMBER_AVAILABLE:
            # Usar pdfplumber se disponível (mais preciso)
            with pdfplumber.open(pdf_path) as pdf:
                for i, pagina in enumerate(pdf.pages[:max_paginas]):
                    print(f"\n📄 PÁGINA {i+1} (pdfplumber):")
                    print("-" * 30)
                    
                    texto = pagina.extract_text()
                    if not texto:
                        print("   ⚠️  Nenhum texto extraído")
                        continue
                    
                    analisar_texto_pagina(texto, i+1)
                    
                    # Tentar extrair tabelas
                    tabelas = pagina.extract_tables()
                    if tabelas:
                        print(f"\n   📊 TABELAS IDENTIFICADAS: {len(tabelas)}")
                        for j, tabela in enumerate(tabelas[:2]):
                            print(f"      Tabela {j+1}: {len(tabela)} linhas, {len(tabela[0]) if tabela else 0} colunas")
        
        else:
            # Fallback para PyPDF2
            reader = PdfReader(pdf_path)
            for i, pagina in enumerate(reader.pages[:max_paginas]):
                print(f"\n📄 PÁGINA {i+1} (PyPDF2):")
                print("-" * 30)
                
                texto = pagina.extract_text()
                if not texto:
                    print("   ⚠️  Nenhum texto extraído")
                    continue
                
                analisar_texto_pagina(texto, i+1)
                
    except Exception as e:
        print(f"   ❌ Erro na análise: {e}")

def analisar_texto_pagina(texto, numero_pagina):
    """Analisa o texto de uma página específica"""
    linhas = texto.split('\n')
    print(f"   📝 Total de linhas: {len(linhas)}")
    
    # Analisar estrutura linha por linha
    print(f"\n   🏗️  LINHAS COM PADRÕES DE MATERIAIS:")
    padroes_encontrados = 0
    
    for num_linha, linha in enumerate(linhas):
        linha_limpa = linha.strip()
        
        # Padrões específicos para materiais de construção
        padroes_materiais = [
            # Códigos de material
            r'M\d{3}',
            # Dimensões
            r'\d+\s*x\s*\d+',
            r'\d+[,.]?\d*\s*m²',
            # Quantidades
            r'\b\d+\s*(?:un|UN|pç|pc|und)',
            # Medidas
            r'\d+\s*cm',
            r'\d+\s*mm',
            # Itens com descrição
            r'(mesa|armário|cadeira|porta|janela|piso|teto|parede)',
        ]
        
        for padrao in padroes_materiais:
            if re.search(padrao, linha_limpa, re.IGNORECASE):
                padroes_encontrados += 1
                print(f"      📍 Linha {num_linha+1}: {linha_limpa[:120]}")
                break
    
    print(f"\n   ✅ Padrões encontrados na página: {padroes_encontrados}")

# Executar análise nos primeiros 2 PDFs
for pdf_path in pdfs[:2]:
    analise_estrutural_manual(pdf_path)


🔍 ANÁLISE ESTRUTURAL MANUAL: Projeto arquitetura Legal Cartão Joinville.pdf

📄 PÁGINA 1 (PyPDF2):
------------------------------
   📝 Total de linhas: 3

   🏗️  LINHAS COM PADRÕES DE MATERIAIS:
      📍 Linha 3: ARQUITETO E URBANISTA ANDRÉ DA SILVA MARTINS -CAU A56427 -3 AUTOR DO PROJETO:ADMINISTRADORA DE CARTÃO DE TODOS DE JOINVI

   ✅ Padrões encontrados na página: 1

📄 PÁGINA 2 (PyPDF2):
------------------------------
   📝 Total de linhas: 143

   🏗️  LINHAS COM PADRÕES DE MATERIAIS:
      📍 Linha 1: M002 -mesa atendimento recepção MDF (135x60/75 cm)
      📍 Linha 2: M003 -mesa tipo baia MDF (80x60/75cm)
      📍 Linha 3: M005 -mesa copa MDF 300X80/75 cm
      📍 Linha 4: M006 -mesa redonda ø 90 cm
      📍 Linha 5: M007 -armário baixo com portas MDF (80x50x75cm)M001 -mesa atendimento MDF (120x60/75 cm)
      📍 Linha 8: E004 -geladeira/frigobarE006 -TVE005 -exaustor tipo ventokit E001 -bebedouro/purificador de águaM004 -longarinas recepçã
      📍 Linha 9: M008 -armários em MDF para arq

Multiple definitions in dictionary at byte 0x674b3 for key /PageMode


   📝 Total de linhas: 96

   🏗️  LINHAS COM PADRÕES DE MATERIAIS:
      📍 Linha 1: A = 5,54 m²ALMOXARIFADOA = 24,72 m²SALA DE VENDAS IIA = 19,66 m²SALA DE VENDAS I
      📍 Linha 2: A = 17,44 m²SALA DE REUNIÕESA = 12,48 m²PRÉ-VENDAS
      📍 Linha 3: A = 20,69 m²ADIMPLÊNCIAA = 3,35 m²SANITÁRIO
      📍 Linha 5: A = 5,21 m²SANITÁRIO
      📍 Linha 7: A = 5,21 m²SANITÁRIO
      📍 Linha 8: MASCULINO A = 4,90 m²DML
      📍 Linha 9: A = 21,25 m²COPA FUNCIONÁRIOS
      📍 Linha 10: A = 43,64 m²RECEPÇÃO
      📍 Linha 11: A = 6,90 m²CONCILIAÇÃO
      📍 Linha 12: A = 5,24 m²ESPERAA = 8,36 m²GERÊNCIA
      📍 Linha 13: M010fechamento em vidro temperado até o forroSobe
      📍 Linha 16: existenteE0011,800,90 0,90E001M013
      📍 Linha 18: existenteA = 5,04 m²FINANCEIRO
      📍 Linha 25: 2,50M010
      📍 Linha 45: A = 10,90 m²ASSESSORES3,80 0,10 2,29 0,13 1,65 0,10 1,50 0,13 1,84 0,10 1,84 0,13 1,76 0,13 1,75 0,13 1,39 0,13 2,95 3,3
      📍 Linha 62: DRYWALL 10cm -A CONSTRUIR
      📍 Linha 63: PAINEL DE

# 🔧 PARSERS ESPECÍFICOS POR TIPO DE ESTRUTURA

In [4]:

class ParserQuantitativos:
    """Parser para extrair materiais de diferentes estruturas"""
    
    def __init__(self):
        self.padroes = self._criar_padroes_especificos()
    
    def _criar_padroes_especificos(self):
        """Cria padrões baseados na análise estrutural"""
        return {
            # Padrão 1: Código - Descrição (Dimensões)
            'codigo_descricao': {
                'regex': r'(M\d{2,4})\s*[-–]?\s*([^-–]+?)\s*(?:\(([^)]+)\))?',
                'campos': ['codigo', 'descricao', 'dimensoes']
            },
            
            # Padrão 2: Item com medidas
            'item_medidas': {
                'regex': r'([^,\.]+?)\s*[,\.]?\s*(\d+)\s*x\s*(\d+)(?:\s*/\s*(\d+))?\s*cm',
                'campos': ['descricao', 'largura', 'altura', 'profundidade']
            },
            
            # Padrão 3: Quantidade + Material
            'quantidade_material': {
                'regex': r'(\d+)\s*(un|UN|pç|pc|und)?\s*[\\-]?\s*([^,\.]+)',
                'campos': ['quantidade', 'unidade', 'material']
            },
            
            # Padrão 4: Áreas
            'areas': {
                'regex': r'[Aa]\s*[=:]?\s*(\d+[,.]?\d*)\s*m²',
                'campos': ['area_m2']
            },
            
            # Padrão 5: Mobiliário completo
            'mobiliario_completo': {
                'regex': r'(M\d{3})\s*-\s*(mesa|armário|cadeira|estante)[^\d]*(\d+)[^\d]*(\d+)',
                'campos': ['codigo', 'tipo', 'dimensao1', 'dimensao2']
            }
        }
    
    def extrair_materiais_estrutura_especifica(self, texto, estrutura='codigo_descricao'):
        """Extrai materiais baseado em estrutura específica"""
        padrao = self.padroes.get(estrutura)
        if not padrao:
            return []
        
        matches = re.finditer(padrao['regex'], texto, re.IGNORECASE)
        materiais = []
        
        for match in matches:
            material = {'estrutura': estrutura}
            for i, campo in enumerate(padrao['campos']):
                if i < len(match.groups()):
                    material[campo] = match.group(i+1)
                else:
                    material[campo] = None
            material['texto_original'] = match.group(0)
            materiais.append(material)
        
        return materiais
    
    def analisar_multiplas_estruturas(self, texto):
        """Tenta múltiplas estruturas e retorna a que encontrar mais matches"""
        resultados = {}
        
        for estrutura in self.padroes.keys():
            materiais = self.extrair_materiais_estrutura_especifica(texto, estrutura)
            if materiais:
                resultados[estrutura] = {
                    'quantidade': len(materiais),
                    'materiais': materiais
                }
        
        return resultados

# Testar o parser
parser = ParserQuantitativos()

print("\n🧪 TESTANDO PARSER ESPECÍFICO:")
print("=" * 40)

def extrair_texto_pdf(pdf_path, max_paginas=2):
    """Extrai texto do PDF usando PyPDF2"""
    try:
        reader = PdfReader(pdf_path)
        texto = ""
        for i, pagina in enumerate(reader.pages[:max_paginas]):
            texto += pagina.extract_text() + "\n"
        return texto
    except Exception as e:
        print(f"   ❌ Erro ao extrair texto: {e}")
        return ""

for pdf_path in pdfs[:1]:  # Testar no primeiro PDF
    print(f"\n📁 Testando em: {pdf_path.name}")
    
    texto = extrair_texto_pdf(pdf_path)
    
    if texto.strip():
        resultados = parser.analisar_multiplas_estruturas(texto)
        
        if resultados:
            print("   ✅ ESTRUTURAS IDENTIFICADAS:")
            for estrutura, dados in resultados.items():
                print(f"      🏗️  {estrutura}: {dados['quantidade']} materiais")
                for material in dados['materiais'][:3]:  # Mostrar 3 exemplos
                    print(f"         📍 {material['texto_original'][:80]}...")
        else:
            print("   ❌ Nenhuma estrutura identificada")
    else:
        print("   ⚠️  Nenhum texto extraído")


🧪 TESTANDO PARSER ESPECÍFICO:

📁 Testando em: Projeto arquitetura Legal Cartão Joinville.pdf
   ✅ ESTRUTURAS IDENTIFICADAS:
      🏗️  codigo_descricao: 58 materiais
         📍 M002 -m...
         📍 M003 -m...
         📍 M005 -m...
      🏗️  item_medidas: 12 materiais
         📍 PROJETO DE ARQUITETURA 
-CARTÃO DE TODOS JOINVILLE -SC -
ARQUITETO E URBANISTA A...
         📍 )
M003 -mesa tipo baia MDF (80x60/75cm...
         📍 )
M005 -mesa copa MDF 300X80/75 cm...
      🏗️  quantidade_material: 74 materiais
         📍 56427 -3 AUTOR DO PROJETO:ADMINISTRADORA DE CARTÃO DE TODOS DE JOINVILLE  LTDA P...
         📍 110x40x180 cm)
M010 -bancada em granito (200x60/90 cm)
E007 -forno microondasLEG...
         📍 00 1...
      🏗️  areas: 19 materiais
         📍 A = 5,54 m²...
         📍 A = 24,72 m²...
         📍 A = 19,66 m²...
      🏗️  mobiliario_completo: 7 materiais
         📍 M002 -mesa atendimento recepção MDF (135x60...
         📍 M003 -mesa tipo baia MDF (80x60...
         📍 M005 -mesa co

# 📝 PROCESSAMENTO INTELIGENTE POR SEÇÕES

In [5]:

def identificar_secoes_relevantes(texto):
    """Identifica seções onde provavelmente estão os quantitativos"""
    
    print("\n🎯 IDENTIFICANDO SEÇÕES RELEVANTES:")
    print("=" * 40)
    
    secoes_chave = [
        'quantitativo', 'material', 'especificação', 'descrição',
        'qtd', 'unidade', 'medidas', 'dimensões', 'mobiliário',
        'lista de materiais', 'composição', 'memorial'
    ]
    
    linhas = texto.split('\n')
    secoes_encontradas = []
    
    for i, linha in enumerate(linhas):
        linha_limpa = linha.strip().lower()
        
        # Verificar se a linha contém palavras-chave de seção
        for secao in secoes_chave:
            if secao in linha_limpa:
                secoes_encontradas.append({
                    'linha': i + 1,
                    'secao': secao,
                    'texto': linha.strip(),
                    'contexto': f"{linhas[i-1] if i>0 else ''} | {linha} | {linhas[i+1] if i<len(linhas)-1 else ''}"
                })
                break
    
    # Agrupar seções próximas
    secoes_agrupadas = []
    if secoes_encontradas:
        current_group = [secoes_encontradas[0]]
        
        for secao in secoes_encontradas[1:]:
            if secao['linha'] - current_group[-1]['linha'] <= 5:  # Seções dentro de 5 linhas
                current_group.append(secao)
            else:
                secoes_agrupadas.append(current_group)
                current_group = [secao]
        
        secoes_agrupadas.append(current_group)
    
    return secoes_agrupadas

def extrair_materiais_por_secao(texto, secoes):
    """Extrai materiais focando nas seções identificadas"""
    
    linhas = texto.split('\n')
    materiais_por_secao = {}
    
    for grupo in secoes:
        # Encontrar range de linhas para a seção
        inicio = max(0, grupo[0]['linha'] - 2)  # 2 linhas antes da seção
        fim = min(len(linhas), grupo[-1]['linha'] + 20)  # 20 linhas depois
        
        texto_secao = '\n'.join(linhas[inicio:fim])
        nome_secao = ' | '.join([s['secao'] for s in grupo])
        
        # Aplicar parser na seção
        parser = ParserQuantitativos()
        resultados = parser.analisar_multiplas_estruturas(texto_secao)
        
        if resultados:
            materiais_por_secao[nome_secao] = {
                'range_linhas': f"{inicio+1}-{fim+1}",
                'resultados': resultados
            }
    
    return materiais_por_secao

# Testar processamento por seções
print("\n📝 PROCESSAMENTO POR SEÇÕES:")
print("=" * 35)

for pdf_path in pdfs[:1]:
    print(f"\n📁 Processando: {pdf_path.name}")
    
    texto_completo = extrair_texto_pdf(pdf_path, max_paginas=3)
    
    if texto_completo.strip():
        # Identificar seções
        secoes = identificar_secoes_relevantes(texto_completo)
        
        if secoes:
            print("   ✅ SEÇÕES IDENTIFICADAS:")
            for grupo in secoes:
                print(f"      📍 {grupo[0]['texto'][:60]}... (linha {grupo[0]['linha']})")
            
            # Extrair materiais por seção
            materiais = extrair_materiais_por_secao(texto_completo, secoes)
            
            if materiais:
                print("\n   🎯 MATERIAIS ENCONTRADOS POR SEÇÃO:")
                for secao, dados in materiais.items():
                    total_materiais = sum(r['quantidade'] for r in dados['resultados'].values())
                    print(f"      🏗️  {secao}: {total_materiais} materiais (linhas {dados['range_linhas']})")
            else:
                print("   ❌ Nenhum material encontrado nas seções")
        else:
            print("   ❌ Nenhuma seção relevante identificada")
    else:
        print("   ⚠️  Nenhum texto extraído")


📝 PROCESSAMENTO POR SEÇÕES:

📁 Processando: Projeto arquitetura Legal Cartão Joinville.pdf

🎯 IDENTIFICANDO SEÇÕES RELEVANTES:
   ✅ SEÇÕES IDENTIFICADAS:
      📍 E007 -forno microondasLEGENDA DE MOBILIÁRIO E EQUIPAMENTOS:... (linha 14)
      📍 287,87 m² ÁREA CONSTRUÍDA TOTAL:* TODAS AS MEDIDAS DEVEM SER... (linha 143)
      📍 0,20ESPECIFICAÇÃO DE MATERIAIS DE ACABAMENTO:... (linha 213)
      📍 TABELA DE PORTAS (dimensões em cm):... (linha 224)
      📍 esmalte sintético* TODAS AS MEDIDAS DEVEM SER CONFERIDAS NO ... (linha 240)

   🎯 MATERIAIS ENCONTRADOS POR SEÇÃO:
      🏗️  mobiliário | mobiliário: 58 materiais (linhas 13-36)
      🏗️  medidas: 1 materiais (linhas 239-244)
      🏗️  especificação: 9 materiais (linhas 212-234)
      🏗️  dimensões: 1 materiais (linhas 223-244)


# 📊 ANÁLISE COMPARATIVA DE ESTRATÉGIAS

In [6]:

def comparar_estrategias_extracao(pdf_path):
    """Compara diferentes estratégias de extração no mesmo PDF"""
    
    print(f"\n📊 COMPARAÇÃO DE ESTRATÉGIAS: {pdf_path.name}")
    print("=" * 50)
    
    estrategias_resultados = {}
    
    texto = extrair_texto_pdf(pdf_path, max_paginas=2)
    
    if not texto.strip():
        print("   ⚠️  Nenhum texto extraído")
        return
    
    # ESTRATÉGIA 1: Regex genéricos (abordagem original)
    padroes_genericos = [
        r'(\d+)\s*(un|UN)\s*[\\-]?\s*([^,\.]+)',
        r'(\d+)\s*x\s*(\d+)\s*cm',
        r'M\d{3}',
    ]
    
    matches_genericos = 0
    for padrao in padroes_genericos:
        matches_genericos += len(re.findall(padrao, texto, re.IGNORECASE))
    
    estrategias_resultados['regex_genericos'] = matches_genericos
    
    # ESTRATÉGIA 2: Parser específico
    parser = ParserQuantitativos()
    resultados_parser = parser.analisar_multiplas_estruturas(texto)
    total_parser = sum(dados['quantidade'] for dados in resultados_parser.values())
    estrategias_resultados['parser_especifico'] = total_parser
    
    # ESTRATÉGIA 3: Processamento por seções
    secoes = identificar_secoes_relevantes(texto)
    materiais_secoes = extrair_materiais_por_secao(texto, secoes)
    total_secoes = sum(
        sum(r['quantidade'] for r in dados['resultados'].values())
        for dados in materiais_secoes.values()
    )
    estrategias_resultados['processamento_secoes'] = total_secoes
    
    # Exibir resultados comparativos
    print("\n   📈 RESULTADOS COMPARATIVOS:")
    for estrategia, resultado in sorted(estrategias_resultados.items(), key=lambda x: x[1], reverse=True):
        status = "✅" if resultado > 0 else "❌"
        print(f"      {status} {estrategia.replace('_', ' ').title()}: {resultado} materiais")
    
    # Recomendação baseada nos resultados
    melhor_estrategia = max(estrategias_resultados, key=estrategias_resultados.get)
    print(f"\n   🎯 RECOMENDAÇÃO: Usar estratégia '{melhor_estrategia.replace('_', ' ').title()}'")
    
    return estrategias_resultados

# Executar comparação para os primeiros 2 PDFs
print("\n" + "=" * 65)
print("🔬 ANÁLISE COMPARATIVA DE ESTRATÉGIAS")
print("=" * 65)

resultados_comparativos = {}
for pdf_path in pdfs[:2]:
    resultados = comparar_estrategias_extracao(pdf_path)
    if resultados:
        resultados_comparativos[pdf_path.name] = resultados


🔬 ANÁLISE COMPARATIVA DE ESTRATÉGIAS

📊 COMPARAÇÃO DE ESTRATÉGIAS: Projeto arquitetura Legal Cartão Joinville.pdf


Multiple definitions in dictionary at byte 0x674b3 for key /PageMode



🎯 IDENTIFICANDO SEÇÕES RELEVANTES:

   📈 RESULTADOS COMPARATIVOS:
      ✅ Parser Especifico: 170 materiais
      ✅ Regex Genericos: 90 materiais
      ✅ Processamento Secoes: 60 materiais

   🎯 RECOMENDAÇÃO: Usar estratégia 'Parser Especifico'

📊 COMPARAÇÃO DE ESTRATÉGIAS: PROJETO AS CANOAS aprovado Bianca 21.07.2022.pdf

🎯 IDENTIFICANDO SEÇÕES RELEVANTES:

   📈 RESULTADOS COMPARATIVOS:
      ✅ Parser Especifico: 169 materiais
      ❌ Regex Genericos: 0 materiais
      ❌ Processamento Secoes: 0 materiais

   🎯 RECOMENDAÇÃO: Usar estratégia 'Parser Especifico'


# 🚀 RESUMO EXECUTIVO E PRÓXIMOS PASSOS

In [9]:

def gerar_resumo_executivo(resultados_comparativos):
    """Gera resumo executivo com base nos resultados"""
    
    print("\n" + "=" * 70)
    print("🚀 RESUMO EXECUTIVO - SOLUÇÕES AVANÇADAS")
    print("=" * 70)
    
    if not resultados_comparativos:
        print("\n❌ Nenhum resultado para análise")
        return
    
    # Análise agregada
    total_pdfs = len(resultados_comparativos)
    estrategias = list(resultados_comparativos.values())[0].keys()
    
    print(f"\n📊 ANÁLISE AGREGADA ({total_pdfs} PDFs):")
    
    for estrategia in estrategias:
        resultados_estrategia = [r[estrategia] for r in resultados_comparativos.values()]
        media = np.mean(resultados_estrategia)
        maximo = max(resultados_estrategia)
        
        print(f"\n   🏗️  {estrategia.replace('_', ' ').title()}:")
        print(f"      • Média: {media:.1f} materiais por PDF")
        print(f"      • Melhor resultado: {maximo} materiais")
        print(f"      • Eficácia: {'ALTA' if media > 10 else 'MÉDIA' if media > 3 else 'BAIXA'}")
    
    # Recomendações finais
    print(f"\n💡 RECOMENDAÇÕES ESTRATÉGICAS:")
    
    recomendacoes = [
        "1. 🎯 IMPLEMENTAR PARSER ESPECÍFICO como solução principal",
        "2. 📝 USAR PROCESSAMENTO POR SEÇÕES para PDFs complexos", 
        "3. 🔧 INSTALAR pdfplumber para extração mais precisa",
        "4. 🏗️ CRIAR PERFIS por tipo de projeto (arquitetura, elétrica, etc.)",
        "5. 📊 IMPLEMENTAR VALIDAÇÃO CRUZADA entre estratégias",
        "6. 🤖 DESENVOLVER APRENDIZADO por projeto para melhorar accuracy"
    ]
    
    for recomendacao in recomendacoes:
        print(f"   {recomendacao}")
    
    print(f"\n🎯 PRÓXIMOS PASSOS IMEDIATOS:")
    print("   • Instalar: !pip install pdfplumber")
    print("   • Implementar pipeline com parser específico")
    print("   • Testar em todos os 16 PDFs disponíveis")
    print("   • Criar sistema de validação manual")

# Gerar resumo
gerar_resumo_executivo(resultados_comparativos)

print(f"\n✅ NOTEBOOK DE SOLUÇÕES AVANÇADAS CONCLUÍDO!")
print("🎯 Próxima etapa: Implementar pipeline de produção")


🚀 RESUMO EXECUTIVO - SOLUÇÕES AVANÇADAS

📊 ANÁLISE AGREGADA (2 PDFs):

   🏗️  Regex Genericos:
      • Média: 45.0 materiais por PDF
      • Melhor resultado: 90 materiais
      • Eficácia: ALTA

   🏗️  Parser Especifico:
      • Média: 169.5 materiais por PDF
      • Melhor resultado: 170 materiais
      • Eficácia: ALTA

   🏗️  Processamento Secoes:
      • Média: 30.0 materiais por PDF
      • Melhor resultado: 60 materiais
      • Eficácia: ALTA

💡 RECOMENDAÇÕES ESTRATÉGICAS:
   1. 🎯 IMPLEMENTAR PARSER ESPECÍFICO como solução principal
   2. 📝 USAR PROCESSAMENTO POR SEÇÕES para PDFs complexos
   3. 🔧 INSTALAR pdfplumber para extração mais precisa
   4. 🏗️ CRIAR PERFIS por tipo de projeto (arquitetura, elétrica, etc.)
   5. 📊 IMPLEMENTAR VALIDAÇÃO CRUZADA entre estratégias
   6. 🤖 DESENVOLVER APRENDIZADO por projeto para melhorar accuracy

🎯 PRÓXIMOS PASSOS IMEDIATOS:
   • Instalar: !pip install pdfplumber
   • Implementar pipeline com parser específico
   • Testar em todos os 16 P